In [88]:
import pandas as pd
import numpy as np
import json as json
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline


### Concatenamos datasets del 2019 y 2020, con datos completos, y del 2021, con información incompleta

In [89]:
data_origin = pd.read_csv(r"C:\Users\Nicolás\Documents\Data Science\Desafio Final\Renzo\Desafio_full.csv",
                          sep=";", encoding='IBM850', low_memory=False)
data_origin = data_origin.drop(7, axis=0)


In [91]:
data_2021 = pd.read_csv(
    r'C:\Users\Nicolás\Documents\Data Science\Desafio Final\Renzo\PARA_PRODUCCION.csv', sep=',', encoding='IBM850')
data_2021.columns = ['NUMERO CLIENTE', 'AÑO', 'Mes', 'Kilos']


In [92]:
for i in data_origin.columns:
    if(i not in data_2021.columns):
        data_2021[i] = np.zeros(len(data_2021))


In [93]:
data_base = pd.concat([data_origin, data_2021])


### Formateamos los datos, obtenemos las devoluciones, filtramos los mercados que nos interesan

In [94]:
class Preprocessing(BaseEstimator, TransformerMixin):
    def __init__(self, to_floats, to_remove, to_remove_markets):
        self.to_floats = to_floats
        self.to_remove = to_remove
        self.to_remove_markets = to_remove_markets

    def to_float_with_extra_steps(self, val):
        try:
            val = val.replace(',', '.')
            return float(val)
        except:
            try:
                return float(val)
            except:
                return np.NaN

    def returns(self, record):
        if((record["Tipo orden"] == "CO") & (record["Kilos"] < 0)):
            record["Devoluciones"] = record['N° Factura']
        else:
            record["Devoluciones"] = np.NaN
        return record

    def payment_condition(self, df):
        keys = df['Condición de pago'].unique()
        values = [30, 15, 45, 30, 60, 45, 90, 45, 30, 60, 0, 0, 7, 30, 75, 45, 60, 21, 60, 90, 110, 120,
                  74, 105, 40, 70, 50, 37.5, 60, 75, 14, 40, 37.5, 105, 0, 15, 1, 75, 20, 80, 120, 30, 45, 15, 90]
        replacement_dict = dict(zip(keys, values))
        df['Condición de pago'] = df['Condición de pago'].replace(
            replacement_dict)
        return df

    def fit(self, X, y=None):
        return self

    def transform(self,  X, y=None):
        X = self.payment_condition(X)
        df_dict = X.to_dict('records')
        df_final = []
        for i in enumerate(df_dict):
            for j in self.to_floats:
                df_dict[i[0]][j] = self.to_float_with_extra_steps(
                    val=(i[1][j]))
            df_dict[i[0]] = self.returns(record=i[1])
            for h in self.to_remove:
                if(df_dict[i[0]][h]):
                    del df_dict[i[0]][h]
            if(i[1]["Mercado"] not in self.to_remove_markets):
                df_final.append(i[1])
        return pd.DataFrame(df_final).sort_values(by=['Mes', 'AÑO'])


### Obtenemos el cuatrimestre de cada registro, agrupamos y calculamos features adicionales

In [106]:
class Quarters_and_grouping(BaseEstimator, TransformerMixin):
    def __init__(self, agg_dict):
        self.agg_dict = agg_dict

    def month_converter(self, df_dict, years):
        # Convierte meses a numeros continuos para el conjunto de años consecutivos
        years.sort()
        for i in enumerate(df_dict):
            df_dict[i[0]]["Mes"] += (12 * years.index(i[1]["AÑO"]))
        return df_dict

    def feature_addition_pre(self, df):

        

        provdict = {}

        df = df.drop(["Moneda", "Pais", "Subrubro"], axis=1)

        cache_df_prov = df.loc[:, ["NUMERO CLIENTE", "Provincia", "Kilos"]]
        cache_df_prov_ = cache_df_prov.groupby(by=["NUMERO CLIENTE", "Provincia"]).agg({
            "Kilos": "sum"}).reset_index()
        
        # Obtiene la Provincia con más Kilos vendidos para cada cliente, y features adicionales
        for i in cache_df_prov_["NUMERO CLIENTE"].unique():
            cache = cache_df_prov_[cache_df_prov_["NUMERO CLIENTE"] == i]
            provincia = cache.loc[cache.Kilos.idxmax(), [
                "Provincia"]].Provincia
            provdict[i] = provincia

        df["Provincia"] = df["NUMERO CLIENTE"].replace(provdict)
        df["Kilos_Promedio"] = df["Kilos"]
        df["Kilos_Maximo"] = df["Kilos"]
        df["Kilos_Minimo"] = df["Kilos"]
        df["Frecuencia"] = df["Mes"]
        df["USD/Ton"] = df.apply(lambda x: (x['USD TC Prom.Mes']/(x['Kilos']/1000)) if ((x['Kilos']
                                                                                         > 100) & (x['USD TC Prom.Mes'] > 100)) else np.nan, axis=1)
        return df

    def quarters(self, df_list, years):
        # Genera cuatrimestres con meses solapados (1 a 4, 2 a 5, etc). Las keys son los meses y los values arrays de cuatrimestres.     
        dfnew = []
        for i in df_list:
            Qs = []
            month = i["Mes"]
            # Genera listas de cuatrimestres con 0 o 1 en cada posición, dependiendo de si el mes pertenece al mismo.
            for q in range(1, (12*len(years)) - 2):
                if((q >= (month - 3)) & (q <= month)):
                    Qs.append(1)
                else:
                    Qs.append(0)
            for v in enumerate(Qs, 0):
                if(v[1]):
                    new = dict(i)
                    quarter = v[0] + 1
                    new.update({"Cuatrimestre": quarter})
                    # Genera columnas con los meses de ese cuatrimestre. Sirve para calcular la target más adelante.
                    new["Mes1"] = 0
                    new["Mes2"] = 0
                    new["Mes3"] = 0
                    new["Mes4"] = 0
                    quarter_position = i["Mes"] - v[0]

                    for m in range(1, 5):
                        new["Mes" + str(m)] = i["Mes"] - (quarter_position - m)

                    del new["Mes"]
                    dfnew.append(new)
        return dfnew

    def grouping(self, df):
        agg = self.agg_dict
        agg["Kilos_Promedio"] = np.mean
        agg["Kilos_Maximo"] = 'max'
        agg["Kilos_Minimo"] = 'min'
        agg["Frecuencia"] = 'nunique'
        agg["USD/Ton"] = np.mean

        for key in ["Mes1", "Mes2", "Mes3", "Mes4"]:
            agg[key] = 'first'

        df_g = df.groupby(
            by=['NUMERO CLIENTE', 'Cuatrimestre']).agg(agg).reset_index()

        df_g_ = df_g[df_g["Kilos"] > 0]
        return df_g_

    def feature_addition_post(self, df):
        df["Indice devoluciones"] = (df['Devoluciones']/df['N° Factura'])
        return df

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        years = list(X["AÑO"].unique())
        df_list = X.to_dict('records')

        data_months_converted = self.month_converter(
            df_dict=df_list, years=years)
        df = pd.DataFrame(data_months_converted)

        data_f = self.feature_addition_pre(df=df)
        df_list = data_f.to_dict('records')

        data_q = self.quarters(df_list=df_list, years=years)

        df_ = pd.DataFrame(data_q)
        data_g = self.grouping(df=df_)

        data_h = self.feature_addition_post(data_g)

        return data_h


### Calculamos la target

In [96]:
class target_definition(BaseEstimator, TransformerMixin):

    def target_calculation(self, df_list):
        target = []
        for i in df_list:
            sum = 0
            for j in df_list:
                if(i["NUMERO CLIENTE"] == j["NUMERO CLIENTE"]):
                    if(np.less(i["Mes4"], j["Mes1"])):
                        if((j["Mes4"] - i["Mes4"]) <= 12):
                            sum += j["Kilos"]
            target.append(int(sum <= 0))
        data_target = pd.DataFrame(df_list)
        data_target["Target"] = target
        return data_target

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        data_list = X.to_dict('records')
        data_target = self.target_calculation(df_list=data_list)
        return data_target


### Eliminamos columnas utilizadas para calcular la target, descartamos los datos del 2021, filtramos outliers y generamos dummies

In [97]:
class Postprocessing(BaseEstimator, TransformerMixin):
    def post_processing(self, df):
        last_month = df["Mes4"].values.max()
        df_ = df[df["Mes4"] < (last_month - 12)]
        df_ = df_.drop(["AÑO", "Mes1", "Mes2", "Mes3",
                       "Mes4", "Mercado"], axis=1)
        df_ = df_[df_["USD/Ton"] < 5000]
        prov_dummies = pd.get_dummies(df_["Provincia"])
        df_ = df_.drop("Provincia", axis=1)

        rubro_dummies = pd.get_dummies(df_["Rubro"])
        df_ = df_.drop("Rubro", axis=1)

        df_f = df_.join(prov_dummies)
        df_f = df_f.join(rubro_dummies)
        df_f.dropna(how='any', inplace=True, axis=0)
        return df_f

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        data_final = self.post_processing(df=X)
        return data_final


### Argumentos para las clases

In [98]:
to_floats = ['NUMERO CLIENTE', 'Línea', 'Cantidad', 'Precio Unitario',
             'Tipo cbio. Prom.Mes', 'Kilos', 'Pesos', 'm2', 'USD TC Prom.Mes']
to_remove = ["Orden", "Remito", "Orden Orig.", "Tipo orden",
             "Período", "Cartón", "Precio total", "m2", "Pesos", "Línea", "TM"]
to_remove_markets = ["MEG", "MIG"]
agg_dict = {'N° Factura': 'nunique', 'SKU': 'nunique', 'Cantidad': np.sum, 'Precio Unitario': np.mean, 'Mercado': 'first', 'Tipo cbio. Prom.Mes': np.mean, 'Kilos': np.sum,
            'USD TC Prom.Mes': np.sum, 'Rubro': 'first',
            'Provincia': 'first', 'Condición de pago': 'max', 'Devoluciones': 'nunique', "AÑO": 'max'}


### Generamos instancias de cada clase

In [99]:
pre = Preprocessing(to_floats=to_floats, to_remove=to_remove,
                    to_remove_markets=to_remove_markets)

data_g = Quarters_and_grouping(agg_dict=agg_dict)

data_target = target_definition()

data_final = Postprocessing()


### Creamos un pipeline con todas las etapas para el entrenamiento del modelo predictor

In [100]:
steps_train_test = [('Preprocessing', pre), ('Quarters and grouping', data_g),
                    ('target_calculation', data_target), ('Postprocessing', data_final)]
pipe = Pipeline(steps_train_test)


In [101]:
data_transformed_model = pipe.fit_transform(data_base)


### Creamos otro, sin cálculo de la target, para los datos que procesará el modelo entrenado

In [102]:
steps_api = [('Preprocessing', pre), ('Quarters and grouping',
                                      data_g), ('Postprocessing', data_final)]
pipe_api = Pipeline(steps_api)


In [103]:
data_api = pipe_api.fit_transform(data_base)


In [107]:
data_transformed_model.to_csv(
    r"C:\Users\Nicolás\Documents\Data Science\Desafio Final\Renzo\data_final.csv", encoding="UTF-8")
